In [ ]:
import tqdm
import glob
import pandas as pd

import pylitreview

from pylitreview import Library
mapLibrary = {"ACM" : Library.ACM,
              "IEEE" : Library.IEEE,
              "ScienceDirect" : Library.ScienceDirect}

from pylitreview import SearchWhere
mapSearchWhere = {"Title" : SearchWhere.Title,
                  "Abstract" : SearchWhere.Abstract,
                  "TitleAbstract" : SearchWhere.TitleAbstract,
                  "Text": SearchWhere.Text}

outputFolderBib = "./files/"

In [ ]:
for fileJob in tqdm.tqdm(glob.glob("./jobs/*.csv"), desc="Jobs"):
    dfJob = pd.read_csv(fileJob)
    dfJob.Done = dfJob.Done.fillna("")
    lstSort = ["Done", "Library", "Key1", "Key2", "YearStart", "YearEnd", "SearchWhere"]
    ascending = [False]
    for x in range(len(lstSort)-1):
        ascending.append(True)
    dfJob = dfJob.sort_values(lstSort, ascending=ascending)
    dfJob.Library = dfJob.Library.map(mapLibrary)
    dfJob.SearchWhere = dfJob.SearchWhere.map(mapSearchWhere)
    
    for i, e in tqdm.tqdm(dfJob.iterrows(), total=len(dfJob), desc="Searches in job"):
        if e.Done != "":
            None
        else:
            infos = e.to_dict()
            keyword = []
            for k in infos.keys():
                if k.startswith('Key'):
                    keyword.append(infos[k])

            infos["Keyword"] = keyword
            success, url = pylitreview.crawl(infos, outputFolderBib)
            dfJob.loc[i, "Url"] = url
            if success:
                dfJob.loc[i, "Done"] = "Done"
                dfJobOut = dfJob.copy()
                dfJobOut.Library = dfJobOut.Library.apply(lambda x: str(x).split(".")[-1])
                dfJobOut.SearchWhere = dfJobOut.SearchWhere.apply(lambda x: str(x).split(".")[-1])
                dfJobOut.to_csv(fileJob, index=False)